<a href="https://colab.research.google.com/github/yoba7/Geomatics/blob/master/improve_perf_of_pip_by_finding_cells_within_shapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google colab environment intialisation

## Missing Linux librairies

We need to install one Ubuntu package. It is a shared library that extends SQLite functionalities. It is the [spatialite](https://en.wikipedia.org/wiki/SpatiaLite) extension.

In [0]:
%%capture

!apt-get install libsqlite3-mod-spatialite

## Python imports

In [0]:
import sqlite3  # Spatialite is a SQLite extension. So we need to import SQLite.
import os
import pandas as pd
import re # re = regular expressions

# Statistical sectors (T_shapes table)

## Download Statistical sectors from StatBel

We can now create our database.

In [0]:
!wget https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_spatialite.zip
!unzip sh_statbel_spatialite.zip
!rm sh_statbel_spatialite.zip

--2019-03-09 15:38:56--  https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_spatialite.zip
Resolving statbel.fgov.be (statbel.fgov.be)... 193.191.245.224
Connecting to statbel.fgov.be (statbel.fgov.be)|193.191.245.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16783727 (16M) [application/zip]
Saving to: ‘sh_statbel_spatialite.zip’

sh_statbel_spatiali 100%[===================>]  16.01M   673KB/s    in 25s     

2019-03-09 15:39:22 (661 KB/s) - ‘sh_statbel_spatialite.zip’ saved [16783727/16783727]

Archive:  sh_statbel_spatialite.zip
replace sh_statbel_statistical_sectors.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sh_statbel_statistical_sectors.sqlite  


## Make a copy of Statistical sectors

We'll work on this copy.

In [0]:
!cp sh_statbel_statistical_sectors.sqlite point2sector_3035.sqlite

## Transform SQLite database into Spatialite database

In [0]:
os.environ["SPATIALITE_SECURITY"]="relaxed" 

connR = sqlite3.connect('point2sector_3035.sqlite')
connR.enable_load_extension(True)
connR.execute('select load_extension("mod_spatialite.so")')   
connR.execute('select InitSpatialMetaData(1);') 
curR = connR.cursor()

## Create a EPSG:3035 version of Statistical sectors (ETRS 89 - LAEA)

In [0]:
curR.execute('''create table T_shapes as
                select cd_sector as id_shape, castToMultipolygon(transform(GeomFromWKB(AsBinary(geometry),31370), 3035))  as geometry
                from sh_statbel_statistical_sectors''')

curR.execute("select recovergeometrycolumn('T_shapes', 'geometry',3035,'MULTIPOLYGON','XY')")
curR.execute("select createSpatialIndex('T_shapes', 'geometry')")

connR.commit()

df=pd.read_sql("select id_shape, st_area(geometry) as ms_area_m2 from T_shapes limit 2",connR)
df.head()

,id_shape,ms_area_m2
0,11001A00-,531588.266569
1,11001A01-,671154.216445


# Create grid (T_grid table)

## Function to encode a cell in Well Known Text (wkt) format

In [0]:
def cellWkt(x,y,l):
    return "POLYGON(("+\
            str(x+0*l)+" "+str(y+0*l)+","+\
            str(x+1*l)+" "+str(y+0*l)+","+\
            str(x+1*l)+" "+str(y+1*l)+","+\
            str(x+0*l)+" "+str(y+1*l)+","+\
            str(x+0*l)+" "+str(y+0*l)+"))"
            
print('Wkt for cell with lower-left corner in ({x},{y}) and length {l} is: {wkt}'.format(x=0,y=0,l=1,wkt=cellWkt(0,0,1)))

Wkt for cell with lower-left corner in (0,0) and length 1 is: POLYGON((0 0,1 0,1 1,0 1,0 0))


## Function to create an empty grid table

In [0]:
def createEmptyGridTableWkt(tableName):
  curR.execute("drop table if exists {}_wkt".format(tableName))
  curR.execute("""create table {}_wkt (
                   parent    text  not null                        ,               
                   id        text  not null unique                 ,
                   x         float not null                        ,
                   y         float not null                        ,
                   wkt       text  not null unique                 )
               """.format(tableName))
  connR.commit()

## Function to transform a grid table with wkt field into grid table with geometry

In [0]:
def wkt2geometry(tableName):
  curR.execute("drop table if exists {}".format(tableName))
  curR.execute("""create table {0} as
                  select parent, id, x, y, geomfromtext(wkt,3035) as geometry
                  from {0}_wkt
               """.format(tableName))
  curR.execute("drop table {}_wkt".format(tableName))
  curR.execute("select recovergeometrycolumn('{}', 'geometry',3035,'POLYGON','XY')".format(tableName))
  curR.execute("select createSpatialIndex('{}', 'geometry')".format(tableName))

## T_shapes bounding bound

In [0]:
df=pd.read_sql("select min(MbrMinX(geometry)),max(MbrMaxX(geometry)),min(MbrMinY(geometry)),max(MbrMaxY(geometry)) as wkt from T_shapes",connR)
df.head()

,min(MbrMinX(geometry)),max(MbrMaxX(geometry)),min(MbrMinY(geometry)),wkt
0,3.799745e+06,4.065361e+06,2.941666e+06,3.167920e+06


## Function to create and populate the 1024m grid table. 

Cells not intersecting shapes are removed from the grid. The grid bbox is about the same as the T_shapes bbox.

In [0]:
def createGrid(xMin,xMax,yMin,yMax):
  '''Create grid with all cells intersecting shapes - this function can be executed one than once without any harm'''  
  step=1024
  myGrid=[] 
    
  for x in range(xMin,xMax+2*step,step):
    for y in range(yMin,yMax+2*step,step):
      myGrid.append(  ( '{step}mN{y}E{x}'.format(x=x,y=y,step=step), x, y, cellWkt(x,y,step) ) )
  
  createEmptyGridTableWkt('T_grid')
  
  curR.executemany("insert into T_grid_wkt (parent,id,x,y,wkt) values ('top',?, ?, ?, ?)",myGrid)

  wkt2geometry('T_grid')

    
  curR.execute(''' create table T_intersectingGridCells as
                   select a.id
                   from T_grid a, T_shapes b
                   where intersects(a.geometry,b.geometry) 
                    and a.rowid in ( 
                                     select rowid 
                                     from spatialIndex
                                     where f_table_name='T_grid' and search_frame=b.geometry
                                    ) ''')
  
  curR.execute("delete from T_grid where id not in (select distinct id from T_intersectingGridCells)")
  curR.execute("drop table T_intersectingGridCells")


  connR.commit()

In [0]:
createGrid(3710*1024,3970*1024,2872*1024,3093*1024)

# Find cells within Shapes


## Usefull functions

### Test for existence of a table in sqlite database

In [0]:
def tableExists(tableName):
    return not pd.read_sql("select * from sqlite_master where type = 'table' and name like '{}'".format(tableName),connR).empty

print(tableExists('T_grid'))
print(tableExists('T_1024m_cells_with_shapes_overlaps'))

True
False


### Get list of table in sqlite database

In [0]:
def tablesList():
    return pd.read_sql("select tbl_name from sqlite_master where type = 'table' and name like 'T_%'",connR)

tablesList()

,tbl_name
0,T_shapes
1,T_grid


### Split a cell

In [0]:
def splitACell(cellLength=None,xParent=None,yParent=None,idParent=None,divideBy=2):
  
  if cellLength % divideBy != 0:
        print("ERROR: cellLength={} and is not a multiple of divideBy={}".format(cellLength,divideBy))
        return
    
  step=int(cellLength/divideBy)

  myCells=[]
  
  for i in range(divideBy):    
    for j in range(divideBy):         
      xCell , yCell = int(xParent+i*step) , int(yParent+j*step) 
      myCells.append((idParent, '{}mN{}E{}'.format(step,yCell,xCell) ,xCell, yCell, cellWkt(xCell,yCell,step)))     

  return myCells

splitACell(cellLength=10,xParent=380,yParent=370,idParent='10mN370E380',divideBy=2)


[('10mN370E380',
  '5mN370E380',
  380,
  370,
  'POLYGON((380 370,385 370,385 375,380 375,380 370))'),
 ('10mN370E380',
  '5mN375E380',
  380,
  375,
  'POLYGON((380 375,385 375,385 380,380 380,380 375))'),
 ('10mN370E380',
  '5mN370E385',
  385,
  370,
  'POLYGON((385 370,390 370,390 375,385 375,385 370))'),
 ('10mN370E380',
  '5mN375E385',
  385,
  375,
  'POLYGON((385 375,390 375,390 380,385 380,385 375))')]

## Create a table containing cells that are *within* a shape

In [0]:
def cellsWithinShape(precision):
  """This function finds cells that are completely within a shape"""

  inputTable='T_{}m_cells_with_shapes_overlaps'.format(precision*2)
  cellsWithinShapes='T_{}m_cells_within_shapes'.format(precision)

  if not tableExists(inputTable):
        inputTable='T_grid'

  curR.execute("drop table if exists {}".format(cellsWithinShapes))
  curR.execute('''create table {cellsWithinShapes} as
                  select a.id, b.id_shape
                  from {cellsWithShapesOverlaps} a, T_shapes b
                  where within(a.geometry,b.geometry) 
                    and a.rowid in ( 
                                     select rowid 
                                     from spatialIndex
                                     where f_table_name='{cellsWithShapesOverlaps}' and search_frame=b.geometry
                                    ) '''.format(cellsWithShapesOverlaps=inputTable,cellsWithinShapes=cellsWithinShapes))
  connR.commit()
  
  return pd.read_sql('select * from {} limit 2'.format(cellsWithinShapes),connR)

## Split cells that overlap a shape

In [0]:
def splitCellsWithShapesOverlaps(precision,divideBy=2):
    
  if precision % divideBy != 0:
    print("ERROR: precision={} and is not a multiple of divideBy={}".format(precision,divideBy))
    return

  inputTable                     ='T_{}m_cells_with_shapes_overlaps'.format(precision*2)

  if not tableExists(inputTable):
        inputTable='T_grid'

  cellsWithinShapes              ='T_{}m_cells_within_shapes'.format(precision)
  cellsWithShapesOverlaps        ='T_{}m_cells_with_shapes_overlaps'.format(precision)

  cellsWithShapesOverlapsDf=pd.read_sql("""select id,x,y
                                           from {} 
                                           where id not in ( select distinct id from {})""".format(inputTable,cellsWithinShapes),connR)

  createEmptyGridTableWkt(cellsWithShapesOverlaps)

  myInserts=[]
  nrec=0

  # Fix: insert only 100.000 records a time in cellsWithShapesOverlaps to avoid memory being exhausted
  for id,x,y in cellsWithShapesOverlapsDf.itertuples(index=False):
    myInserts.extend(splitACell(cellLength=precision,xParent=x,yParent=y,idParent=id,divideBy=divideBy))
    nrec+=1
    if nrec>100000:
        curR.executemany("insert into {}_wkt (parent,id,x,y,wkt) values (?, ?, ?, ?, ?)".format(cellsWithShapesOverlaps),myInserts)
        connR.commit()
        nrec=0
        myInserts=[]
    
    
  curR.executemany("insert into {}_wkt (parent,id,x,y,wkt) values (?, ?, ?, ?, ?)".format(cellsWithShapesOverlaps),myInserts)
  connR.commit()
  
  wkt2geometry(cellsWithShapesOverlaps)

  return pd.read_sql('select parent, id, x, y from {} limit 2'.format(cellsWithShapesOverlaps),connR)

## Identify cells that are within shape and split cells that overlap in order to make them fall within shape

In [0]:
cellsWithinShape(1024)

,id,id_shape
0,1024mN3141632E3930112,11002K171
1,1024mN3142656E3924992,11002K172


In [0]:
splitCellsWithShapesOverlaps(1024)

,parent,id,x,y
0,1024mN3133440E3799040,512mN3133440E3799040,3799040.0,3133440.0
1,1024mN3133440E3799040,512mN3133952E3799040,3799040.0,3133952.0


In [0]:
cellsWithinShape(512)
splitCellsWithShapesOverlaps(512)

cellsWithinShape(256)

"""
splitCellsWithShapesOverlaps(256)

cellsWithinShape(128)
splitCellsWithShapesOverlaps(128)

cellsWithinShape(64)
splitCellsWithShapesOverlaps(64)

cellsWithinShape(32)
splitCellsWithShapesOverlaps(32)

cellsWithinShape(16)
splitCellsWithShapesOverlaps(16)

cellsWithinShape(8)
"""

'\nsplitCellsWithShapesOverlaps(256)\n\ncellsWithinShape(128)\nsplitCellsWithShapesOverlaps(128)\n\ncellsWithinShape(64)\nsplitCellsWithShapesOverlaps(64)\n\ncellsWithinShape(32)\nsplitCellsWithShapesOverlaps(32)\n\ncellsWithinShape(16)\nsplitCellsWithShapesOverlaps(16)\n\ncellsWithinShape(8)\n'

# Export results

In [0]:
from google.colab import files

def exportTable(precision):
  pd.read_sql("select * from T_{}m_cells_within_shapes".format(precision),connR).to_csv('T_{}m_cells_within_shapes.csv'.format(precision),sep='|',index=False)
  
  

In [0]:
exportTable(1024)
exportTable(512)
exportTable(256)
#exportTable(128)
#exportTable(64)
#exportTable(32)
#exportTable(16)
#exportTable(8)

In [0]:
!zip T_cells_within_shapes.zip T_*_cells_within_shapes.csv

updating: T_1024m_cells_within_shapes.csv (deflated 80%)
updating: T_256m_cells_within_shapes.csv (deflated 86%)
updating: T_512m_cells_within_shapes.csv (deflated 84%)


In [0]:
!rm T_*_cells_within_shapes.csv

In [0]:
from google.colab import files

files.download('T_cells_within_shapes.zip')